In [4]:
import base64
print(base64.b85decode(b"bZBKDW^7?+AZc?TZ*CxTXk{R7Wq5QTY-w&~Iv@%lAZBb~XM0yjG*n+SUrjS4HGK").decode())

the flag is on the next line: 
  flag{WH4T_4_M3$5}


In [1]:
@login_required
@app.route('/request_flag')
def request_flag():
     user: User = load_user(current_user.id)
        if user.username == 'user0':
            task = Task.query.filter_by(name='backdoor').first()
            return redirect(url_for('.index', message_type='success', message=task.flag))
         else:
             return redirect(url_for('.index', message_type='fail', message="Sorry, I can't give you the flag"))

SyntaxError: invalid character in identifier (<ipython-input-1-e69ee0671f44>, line 4)

In [33]:
import csv
import flask

from flask import Flask, Response, redirect, render_template, request, url_for
from flask_login import LoginManager, UserMixin, login_user, logout_user, login_required, current_user
from flask_sqlalchemy import SQLAlchemy

from typing import *

from secret import secret, database_uri, tasks_csv_path

app = Flask(__name__)

login_manager = LoginManager()
login_manager.init_app(app)

app.config['SECRET_KEY'] = secret
app.config['SQLALCHEMY_DATABASE_URI'] = database_uri

db = SQLAlchemy(app)


class User(db.Model, UserMixin):                                              # user: log in
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    username = db.Column(db.String(80), nullable=False, unique=True)
    password = db.Column(db.String(80), nullable=False)
    score = db.Column(db.Integer, nullable=False, default=0)

    def __init__(self, username: str, password: str):
        self.username = username
        self.password = password


class Task(db.Model):                                                          # task fulfillment: flag check mark
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    name = db.Column(db.String(10), nullable=False, unique=True)
    flag = db.Column(db.String(80), nullable=False, unique=True)
    full_mark = db.Column(db.Integer, nullable=False)
    num_solved = db.Column(db.Integer, nullable=False, default=0)

    def __init__(self, name: str, flag: str, full_mark: int):
        self.name = name
        self.flag = flag
        self.full_mark = full_mark


class Submission(db.Model):                                                         # submission
    id = db.Column(db.Integer, primary_key=True, autoincrement=True)
    user_id = db.Column(db.Integer, nullable=False)
    task_id = db.Column(db.Integer, nullable=False)

    def __init__(self, user_id: int, task_id: int):
        self.user_id = user_id
        self.task_id = task_id


@login_manager.user_loader                                                        # load
def load_user(user_id) -> User:                        
    return User.query.get(user_id)


@app.route('/', methods=['GET'])
def index() -> Union[Response, str]:
    if current_user.is_authenticated:
        user: User = load_user(current_user.id)
    else:
        return redirect('/login')

    return render_template(
        'index.html',
        username=user.username,
        score=str(user.score),
        rank=str(User.query.filter(User.score > user.score).count() + 1),
        message=request.args.get('message'),
        message_type=request.args.get('message_type'),
    )


@app.route('/login', methods=['GET', 'POST'])
def login() -> Union[Response, str]:
    if flask.request.method == 'GET':
        return render_template('login.html', message='')

    try:
        user = User.query.filter_by(username=request.form['username']).first()

        if user is not None:
            if user.password == request.form['password']:
                login_user(user)
                return redirect(url_for('.index'))
            else:
                return render_template('login.html', message="Wrong password")
        else:
            return render_template('login.html', message="User does not exist")

    except Exception as ex:
        return render_template('login.html', message=ex)


@login_required
@app.route('/logout')
def logout() -> Response:
    logout_user()
    return redirect('/')


@app.route('/register', methods=['GET', 'POST'])
def register() -> Union[Response, str]:
    if flask.request.method == 'GET':
        return render_template('register.html')

    try:
        if User.query.filter_by(username=request.form['username']).first() is not None:
            return render_template('register.html', message="Username already exists")
        elif request.form['username'] == "" or request.form['password'] is None:
            return render_template('register.html', message="Username and password cannot be empty")

        user = User(request.form['username'], request.form['password'])
        db.session.add(user)
        db.session.commit()
        login_user(user)
        return redirect(url_for('.index'))

    except Exception as ex:
        print(ex)
        return render_template('register.html', message=ex)


@app.route('/download/<task>', methods=['GET'])
async def download_task_data(task: str) -> Response:
    return app.send_static_file('data/' + task + '.zip')


@app.route('/source', methods=['GET'])
async def get_source() -> Response:
    resp = flask.Response(open(__file__, encoding='utf-8').read())
    resp.headers['content-type'] = 'text/plain; charset=utf-8'
    return resp


@login_required
@app.route('/submit/<task>', methods=['POST'])
def submit_flag(task: str) -> Response:
    flag = request.form['flag']

    try:
        user_id = current_user.id
        user: User = load_user(user_id)
        task: Task = Task.query.filter_by(name=task).first()

        if task.flag == flag:
            score = int((1 - min((0.5 * task.num_solved / 10), 1)) * task.full_mark)
            if Submission.query.filter_by(user_id=user.id).filter_by(task_id=task.id).first() is None:
                User.query.filter_by(id=user.id).update({'score': user.score + score if score > 100 else 100})
                Task.query.filter_by(id=task.id).update({'num_solved': task.num_solved + 1})
                db.session.add(Submission(user.id, task.id))
                db.session.commit()
            return redirect(url_for('.index', message_type='success', message="Correct flag!"))
        else:
            return redirect(url_for('.index', message_type='fail', message="Wrong flag!"))

    except Exception as ex:
        print(ex)
        return redirect(url_for('.index', message_type='fail', message="Error"))


@login_required
@app.route('/request_flag')                  # key step
def request_flag():                                                                          #important message
    user: User = load_user(current_user.id)
    if user.username == 'user0':
        task = Task.query.filter_by(name='backdoor').first()
        return redirect(url_for('.index', message_type='success', message=task.flag))
    else:
        return redirect(url_for('.index', message_type='fail', message="Sorry, I can't give you the flag"))


@app.route('/rank')
def rank_list():                                                                      #rank table

    user_info_list = list()

    for user in User.query.all():
        user_info = dict()
        user_info['name'] = user.username
        user_info['score'] = user.score
        user_info['tasks'] = list()

        for task in Task.query.all():
            if Submission.query.filter(Submission.user_id == user.id).filter(Submission.task_id == task.id).count() > 0:
                user_info['tasks'].append((task.name, True))
            else:
                user_info['tasks'].append((task.name, False))

        if any(t[1] for t in user_info['tasks']):
            user_info_list.append(user_info)

    user_info_list.sort(key=lambda u: u['score'], reverse=True)
    return render_template('rank.html', user_info=user_info_list)


@app.before_first_request
def init_database():
    db.create_all()

    if User.query.filter_by(username='user0').first() is None:
        user = User('user0', 'Default.User#0')                                     # password is Default.User#0 !
        db.session.add(user)
        db.session.commit()
                                                         
    try:
        with open(tasks_csv_path, newline='') as tasks_csv_file:
            tasks_csv_reader = csv.DictReader(tasks_csv_file)

            for task_row in tasks_csv_reader:
                task_name = task_row['name']
                flag = task_row['flag']
                full_mark = int(task_row['mark'])
                if Task.query.filter_by(name=task_name).first() is None:
                    task = Task(task_name, flag, full_mark)
                    db.session.add(task)

    except Exception as ex:
        print(ex)
        exit(-1)

    db.session.commit()

# if __name__ == '__main__':
#     app.run(host='0.0.0.0', debug=True)

SyntaxError: invalid syntax (tasks.py, line 565)

backdoor
flag{N0_SY5T3M_15_4B$0LUT3LY_5AF3}

In [ ]:
from gmpy2 import *
import time
start=time.clock()
n = mpz(63281217910257742583918406571)
x = mpz(2)
y = x**2 + 1
for i in range(n):
  p = gcd(y-x,n)
  if p != 1:
    print(p)
    break
  else:
    y=(((y**2+1)%n)**2+1)%n
    x=(x**2+1)%n
end=time.clock()
print(end-start)

In [1]:
➜  RsaCtfTool-master python3 RsaCtfTool.py --publickey /mnt/c/Users/blue/Desktop/workspace/pubkey.pem --uncipherfile /mnt/c/Users/blue/Desktop/workspace/flag.enc
[+] Clear text : b'\x00\x02\xc0\xfe\x04\xe3&\x0e[\x87\x00PCTF{256b_i5_m3dium}\n'

SyntaxError: invalid character in identifier (<ipython-input-1-8ff84899e12c>, line 1)

In [2]:
RsaCtfTool-master python3 RsaCtfTool.py --dumpkey --key /mnt/c/Users/blue/Desktop/workspace/pubkey.pem
[*] n: 87924348264132406875276140514499937145050893665602592992418171647042491658461
[*] e: 2

➜  RsaCtfTool-master cat /mnt/c/Users/blue/Desktop/workspace/flag.enc
9╚m'WiK?GxC7H%

SyntaxError: invalid syntax (<ipython-input-2-30378ee4b33c>, line 1)

In [13]:

import z3
import gmpy2
from Crypto.Util.number import *
s = 271672999328694663443089640948693188141881540359129571214427156310348351584771573522812810966104706380403880838365722908269776661929937802923497894615942297243639275941824176499196943467021517392290120490427967981055467614071328075349003277951520207307790709219332788534534059469701408571793111801600089332398
n = 18244658881181295319272954883581678165861403741755390442222821758115654615383597556932534821731714810350902659365958634733959171693227147622579104808443800023012807714693188254396658989969487167715220145731741176715085310151537361663088589594993290952472481178948462183192763514126228103186090214442977240685491317488226909051593066670516048910819151311106270034153401256626490170049142121264758358176294786363098332982684729097021211531812844797607301714352763239886641744402910016721377618233849598049421731283613204896542399123531691287972535075680132369125078503060130096219979805250532197494614877651313570079377
c = 9982404430128628319460096817021682708004041565035290186717543367896053768314546561325519047732421305742915803158627731630816806760457896955185660519641210406070992709854525079205889290656766462542565382082500517018022078935175455174193675834049195975249215857769348493749181265812457145268873504426358369635240359831717983659211576843033366734030629007224954735338968474959579208259275500679834724272930952006386468745205010136829227065466809167172815459266107445250332546484781883749881174230824790424174571269001813440684711977243960629870372992760938447771175944247778848268805750052191141866322609455381738837998
so=z3.Solver()
p,q=z3.Ints("p q")

so.add(p+q==s)
so.add(p*q==n)
so.add(p>0)
arr=[]

#校验是否有解

if so.check()==z3.sat:

    arr = so.model()

print(arr)

#得到p，q

[q = 121452628145144393423317809328333425215257320416746185773469993658994820464022160054386506840537083750682040109301039308860847871253835883403612894665217710146877290825749023533787465714022707712660546674992877210389908097097449729525173802237962699091767935256681160094155787379477623246951290464087946041031,
 p = 150220371183550270019771831620359762926624219942383385440957162651353531120749413468426304125567622629721840729064683599408928790676101919519884999950724587096761985116075152965409477752998809679629573815435090770665559516973878345823829475713557508216022773962651628440378272090223785324841821337512143291367]


[q = 121452628145144393423317809328333425215257320416746185773469993658994820464022160054386506840537083750682040109301039308860847871253835883403612894665217710146877290825749023533787465714022707712660546674992877210389908097097449729525173802237962699091767935256681160094155787379477623246951290464087946041031,
 p = 150220371183550270019771831620359762926624219942383385440957162651353531120749413468426304125567622629721840729064683599408928790676101919519884999950724587096761985116075152965409477752998809679629573815435090770665559516973878345823829475713557508216022773962651628440378272090223785324841821337512143291367]

 

In [15]:
import gmpy2
q = 121452628145144393423317809328333425215257320416746185773469993658994820464022160054386506840537083750682040109301039308860847871253835883403612894665217710146877290825749023533787465714022707712660546674992877210389908097097449729525173802237962699091767935256681160094155787379477623246951290464087946041031
p = 150220371183550270019771831620359762926624219942383385440957162651353531120749413468426304125567622629721840729064683599408928790676101919519884999950724587096761985116075152965409477752998809679629573815435090770665559516973878345823829475713557508216022773962651628440378272090223785324841821337512143291367


m = gmpy2.lcm(q-1, p-1)
print(m)


3040776480196882553212159147263613027643567290292565073703803626352609102563932926155422470288619135058483776560993105788993195282204524603763184134740633337168801285782198042399443164994914527952536690955290196119180885025256226943848098265832215158745413529824743697198793919021038017197681702407162873447536607414816369064691662838261225953779544961791190077156495683386023636410728424623657591201698346663782408690719834364791905811647151165780633969956136823773833744743514306703696779127804680109521935132197539485914488584910060535437255299621435360302881298973466217947574290963471798153803627641618913457830


In [16]:

e = 65537
n = 18244658881181295319272954883581678165861403741755390442222821758115654615383597556932534821731714810350902659365958634733959171693227147622579104808443800023012807714693188254396658989969487167715220145731741176715085310151537361663088589594993290952472481178948462183192763514126228103186090214442977240685491317488226909051593066670516048910819151311106270034153401256626490170049142121264758358176294786363098332982684729097021211531812844797607301714352763239886641744402910016721377618233849598049421731283613204896542399123531691287972535075680132369125078503060130096219979805250532197494614877651313570079377
c = 9982404430128628319460096817021682708004041565035290186717543367896053768314546561325519047732421305742915803158627731630816806760457896955185660519641210406070992709854525079205889290656766462542565382082500517018022078935175455174193675834049195975249215857769348493749181265812457145268873504426358369635240359831717983659211576843033366734030629007224954735338968474959579208259275500679834724272930952006386468745205010136829227065466809167172815459266107445250332546484781883749881174230824790424174571269001813440684711977243960629870372992760938447771175944247778848268805750052191141866322609455381738837998
m = 3040776480196882553212159147263613027643567290292565073703803626352609102563932926155422470288619135058483776560993105788993195282204524603763184134740633337168801285782198042399443164994914527952536690955290196119180885025256226943848098265832215158745413529824743697198793919021038017197681702407162873447536607414816369064691662838261225953779544961791190077156495683386023636410728424623657591201698346663782408690719834364791905811647151165780633969956136823773833744743514306703696779127804680109521935132197539485914488584910060535437255299621435360302881298973466217947574290963471798153803627641618913457830
d = pow(e, -1, m)                            #pow(x,y,z)：这个是表示x的y次幂后除以z的余数。求出d即可解密

FLAG = pow(c, d, n)      
print(FLAG) 

42134526936703520298201966791804502061510425695548421959165693565


In [17]:
c = 9982404430128628319460096817021682708004041565035290186717543367896053768314546561325519047732421305742915803158627731630816806760457896955185660519641210406070992709854525079205889290656766462542565382082500517018022078935175455174193675834049195975249215857769348493749181265812457145268873504426358369635240359831717983659211576843033366734030629007224954735338968474959579208259275500679834724272930952006386468745205010136829227065466809167172815459266107445250332546484781883749881174230824790424174571269001813440684711977243960629870372992760938447771175944247778848268805750052191141866322609455381738837998





In [31]:
FLAG = 42134526936703520298201966791804502061510425695548421959165693565

import binascii as b
print(hex(FLAG))



0x666c61677b5231563335545f5348344d31525f41444c334d344e7d


b'\x92TIG"D\x90f0pW\t &g\x91q!Q\x87w\x03\x15t\x80\x00d\x02#)\x80Qb\x108Vtu\x06bT\x06ei\x15\x10sw\x83\x83\x11FQD\x12a\x00\x91D\x00p6\x18\'B\x92\x07)U\x19\x12\x86\x94\x90\x95\x18\x03r\x85\x95\x83\x91RpU\x08\x927S\x0549PP\x02t\x882\x11XtQe\x00\x93PY\x97xh\x95\x18"\x011Rr\t@b\x08xRPb x\x855\x96s"9\x014`\x07\x12eYw&\x10\x87\x99$p$\x1921Qfr\x15T\x95&T\x03x\x02\x93x\x88u\x18\x82\x80e\x81\x83\x15\x899#\x12d\x14S`y5f8BIG\x861Y\x05G5\'g\x14\x84\'\x08!(F\x91\x87\x15\x83%\x86\x96x3\x95\x88\x89%\x91G\x81\x19!0\x14\x95B\x17\x97v\x98e\x99aB\x12\x83&3\x82V\t\x86I\x99\x04\x08\x04Y!\x98%UxP%2\x13"d#\x96 \x13#(uu\x03\t\x02\tG @\x89b\x81Sd\x81F\x89bUaI\x08\x96)xd\x91\x06\x19\x03\x98(\x17Dc\x870 Bv\t\x87\x94#'

In [33]:
zam = 0x666c61677b5231563335545f5348344d31525f41444c334d344e7d
encoding='ISO-8859-9'
ans = zam.decode(encoding)

print(ans)

AttributeError: 'int' object has no attribute 'decode'

TIG"Df0pW	 &gq!Qwtd#)Qb8VtubTeiswFQDaDp6'B)UrRp7S49PPt2XtQePYxh"1Rr	@xRPb x5s"94`eYw&$p$21QfrT&Txxue9#dS`y5f8BIG1YG5'g!(F%x3%G!0BveaB&3V	IY!%UxP%2"d# #(uu		G @bSdFbUa)xd(Dc0 Bv	#